# Main imports and code

In [30]:
import sys
sys.path.insert(0, '..')

from scripts.pcl.dont_patronize_me import DontPatronizeMe

DATASET_DIR_PCL = '../dataset/pcl'

In [ ]:
# check which gpu we're using
!nvidia-smi

Mon Nov 22 21:21:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
%%capture
!pip install simpletransformers
!pip install tensorboardx

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached simpletransformers-0.63.3-py3-none-any.whl (247 kB)
  Using cached sentencepiece-0.1.96-cp39-cp39-macosx_10_6_x86_64.whl (1.1 MB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached streamlit-1.2.0-py2.py3-none-any.whl (9.1 MB)
  Using cached wandb-0.12.7-py2.py3-none-any.whl (1.7 MB)
  Using cached psutil-5.8.0-cp39-cp39-macosx_10_9_x86_64.whl (236 kB)
  Using cached configparser-5.2.0-py3-none-any.whl (19 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached GitPython-3.1.24-py3-none-any.whl (180 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)


  Using cached altair-4.1.0-py3-none-any.whl (727 kB)
  Using cached base58-2.1.1-py3-none-any.whl (5.6 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached blinker-1.4-py3-none-any.whl


  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached grpcio-1.42.0-cp39-cp39-macosx_10_10_x86_64.whl (4.0 MB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.c

  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no

In [23]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval

In [24]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  False


In [25]:
if cuda_available:
    import tensorflow as tf
    # Get the GPU device name.
    device_name = tf.test.gpu_device_name()
    # The device name should look like the following:
    if device_name == '/device:GPU:0':
        print('Found GPU at: {}'.format(device_name))
    else:
        raise SystemError('GPU device not found')

# Fetch Don't Patronize Me! data manager module

In [1]:
#module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
#module_name = module_url.split('/')[-1]
#print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
#with request.urlopen(module_url) as f, open(module_name,'w') as outf:
#  a = f.read()
#  outf.write(a.decode('utf-8'))

In [26]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [31]:
dpm = DontPatronizeMe(DATASET_DIR_PCL, None)

In [32]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Load paragraph IDs

In [33]:
trids = pd.read_csv(os.path.join(DATASET_DIR_PCL, 'practice_splits', 'train_semeval_parids-labels.csv'))
teids = pd.read_csv(os.path.join(DATASET_DIR_PCL, 'practice_splits', 'dev_semeval_parids-labels.csv'))

In [34]:
trids.head()

,par_id,label
0,4341,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,"[1, 0, 0, 1, 1, 1, 0]"


In [35]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

# Rebuild training set (Task 1)

In [36]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
    parid = trids.par_id[idx]
    #print(parid)
    # select row from original dataset to retrieve `text` and binary label
    text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
    label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
    rows.append({
        'par_id':parid,
        'text':text,
        'label':label
    })
  

In [49]:
trdf1 = pd.DataFrame(rows)
trdf1.tail()

,par_id,text,label
2089,10462,"the sad spectacle , which occurred on saturday...",0
2090,10463,""""""" the pakistani police came to our house and...",0
2091,10464,"""when marie o'donoghue went looking for a spec...",0
2092,10465,"""sri lankan norms and culture inhibit women fr...",0
2093,10466,he added that the afp will continue to bank on...,0


# Rebuild test set (Task 1)

In [38]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
    parid = teids.par_id[idx]
    #print(parid)
    # select row from original dataset
    text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
    label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
    rows.append({
        'par_id':parid,
        'text':text,
        'label':label
    })
  

In [39]:
len(rows)

2094

In [48]:
tedf1 = pd.DataFrame(rows)
tedf1.tail()

,par_id,text,label
2089,10462,"the sad spectacle , which occurred on saturday...",0
2090,10463,""""""" the pakistani police came to our house and...",0
2091,10464,"""when marie o'donoghue went looking for a spec...",0
2092,10465,"""sri lankan norms and culture inhibit women fr...",0
2093,10466,he added that the afp will continue to bank on...,0


# RoBERTa Baseline for Task 1

In [45]:
# positive vs negative sample in original dataset
len(trdf1[trdf1.label==1]), len(trdf1[trdf1.label==0])

(794, 7581)

In [43]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf, trdf1[trdf1.label==0][: npos*2]])

In [44]:
# positive vs negative sample in training dataset after downsampling negative instances
len(training_set1[training_set1.label==1]), len(training_set1[training_set1.label==0])

(794, 1588)

In [ ]:

task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set1[['text', 'label']])
# run predictions
preds_task1, _ = task1_model.predict(tedf1.text.tolist())

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/298 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
Counter(preds_task1)

Counter({0: 1651, 1: 443})

In [ ]:
!mkdir {ref,res}

In [ ]:
# store reference labels
labels2file(tedf1.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task1.txt'))
# store predicted labels
labels2file([[k] for k in preds_task1], os.path.join('res/', 'task1.txt'))

## Prepare submission

In [ ]:
!cat task1.txt | head -n 10

1
1
0
1
0
0
1
1
0
1


In [ ]:
!zip submission.zip task1.txt

  adding: task1.txt (deflated 92%)
  adding: task2.txt (deflated 97%)
